In [1]:
import albumentations as A
import numpy as np

import matplotlib.pyplot as plt
from torchvision.datasets import Caltech256 ,Caltech101 ,CIFAR100
import os
from PIL import Image
from urllib.request import urlretrieve
import requests
import tarfile

In [2]:
from torch import nn

In [3]:
import torch

In [ ]:
# With square kernels and equal stride
c = nn.Conv2d(16, 33, 3, stride=2)
# non-square kernels and unequal stride and with padding
m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))
d = nn.Conv2d(3,44,4)
a=[]
kk =[c,m]
dd = [d]
a += kk
a += dd
# li = nn.Sequential(*a)

li = nn.ModuleList(a)

# non-square kernels and unequal stride and with padding and dilation
# m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1))
input = torch.ones(20, 16, 50, 100)
output = m(input)



In [27]:
t = torch.tensor([[[1, 2],
                   [3, 4]],
                  [[5, 6],
                   [7, 8]]])
torch.flatten(t)
torch.flatten(t, start_dim=1)

tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]])

In [28]:
t.shape

torch.Size([2, 2, 2])

In [25]:
tt = torch.ones(4,1000,1,1)

tt=  torch.flatten(tt,start_dim = 1)

In [26]:
tt.shape

torch.Size([4, 1000])

In [ ]:
li

ModuleList(
  (0): Conv2d(16, 33, kernel_size=(3, 3), stride=(2, 2))
  (1): Conv2d(16, 33, kernel_size=(3, 5), stride=(2, 1), padding=(4, 2))
  (2): Conv2d(3, 44, kernel_size=(4, 4), stride=(1, 1))
)

In [ ]:
output.shape

torch.Size([20, 33, 28, 100])

## VGG model define

In [4]:
Config_channels = {
"A" : [64,"M" , 128,  "M"  , 256,256,"M" ,512,512 ,"M" , 512,512,"M"] ,
"A_lrn" : [64,"LRN","M" , 128,  "M"  , 256,256,"M" ,512,512 ,"M" , 512,512,"M"] ,
"B" :[64,64,"M" , 128,128,  "M"  , 256,256,"M" ,512,512 ,"M" , 512,512,"M"]  ,
"C" : [64,64,"M" , 128,128,  "M"  , 256,256,256,"M" ,512,512 ,512,"M" , 512,512,512,"M"] ,
"D" :[64,64,"M" , 128,128,  "M"  , 256,256,256,"M" ,512,512 ,512,"M" , 512,512,512,"M"] ,
"E" :[64,64,"M" , 128,128,  "M"  , 256,256,256,256,"M" ,512,512 ,512,512,"M" , 512,512,512,512,"M"]         ,

}

In [5]:
Config_kernel = {
"A" : [3,2 , 3,  2  , 3,3,2 ,3,3 ,2 , 3,3,2] ,
"A_lrn" : [3,2,2 , 3,  2  , 3,3,2 ,3,3 ,2 , 3,3,2] ,
"B" :[3,3,2 , 3,3,  2  , 3,3,2 ,3,3 ,2 , 3,3,2]  ,
"C" : [3,3,2 , 3,3,  2  , 3,3,1,2 ,3,3 ,1,2 , 3,3,1,2] ,
"D" :[3,3,2 , 3,3,  2  , 3,3,3,2 ,3,3 ,3,2 , 3,3,3,2] ,
"E" :[3,3,2 , 3,3,  2  , 3,3,3,3,2 ,3,3 ,3,3,2 , 3,3,3,3,2]         ,

}

In [38]:
def make_feature_extractor(cfg_c,cfg_k):
    feature_extract = []
    in_channels = 3
    i = 1
    for  out_channels , kernel in zip(cfg_c,cfg_k) :
        # print(f"{i} th layer {out_channels} processing")
        if out_channels == "M" :
            feature_extract += [nn.MaxPool2d(kernel,2) ]
        elif out_channels == "LRN":
            feature_extract += [nn.LocalResponseNorm(5,k=2) , nn.ReLU()]
        else :
            feature_extract+= [nn.Conv2d(in_channels,out_channels,kernel,stride = 1 , padding = 1) , nn.ReLU()]

        if isinstance(out_channels,int) :   in_channels = out_channels
        i+=1
    return nn.Sequential(*feature_extract)


In [41]:
class Model_vgg(nn.Module) :
    def __init__(self,version , num_classes):
        conv_5_out_w ,conv_5_out_h = 7,7
        conv_5_out_dim =512
        conv_1_by_1_1_outchannel = 4096
        conv_1_by_1_2_outchannel = 4096
        # conv_1_by_1_3_outchannel = num_classes
        super().__init__()
        self.feature_extractor = make_feature_extractor(Config_channels[version] , Config_kernel[version])

        self.output_layer = nn.Sequential(
                             nn.Conv2d(conv_5_out_dim  ,conv_1_by_1_1_outchannel ,7) ,
                             nn.ReLU(),
                             nn.Dropout(),
                             nn.Conv2d(conv_1_by_1_1_outchannel ,conv_1_by_1_2_outchannel,1 ) ,
                             nn.ReLU(),
                             nn.Dropout(),
                             nn.Conv2d(conv_1_by_1_2_outchannel ,num_classes,1 )
                             )

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))

    def forward(self,x):
        x = self.feature_extractor(x)
        x = self.output_layer(x)
        x= self.avgpool(x)
        x= torch.flatten(x,start_dim = 1)
        return x


    #     pass



## Unit Test

### Model Parameter Unit Test

In [20]:
model_version = ["A","A_lrn" , "B","C","D","E"]

In [17]:
model_param_num  = [133,133,133,134,138,144]

In [11]:
import math


In [8]:
def parameter_num(model_parameters):
    summed = sum([torch.numel(p) for p in model_parameters])
    return summed

In [42]:
for v ,number in zip(model_version ,model_param_num):
    num_classes= 1000
    model_test = Model_vgg(v,num_classes)
    print(f"{v} model processing")
    print(round(parameter_num(model_test.parameters()) / 1e+6))
    assert(round(parameter_num(model_test.parameters()) / 1e+6) == number)

A model processing
133
A_lrn model processing
133
B model processing
133
C model processing
134
D model processing
138
E model processing
144


### Model outputshape test

In [43]:
random_tensor = torch.rand((8,3,389,389))

model_test = Model_vgg("A" , 1000)

In [47]:
random_shape =[(9,3,400,400)  ,  (1,3,400,400)  ,(1,3,224,224)  , (8,3,389,389)]

In [50]:
for sp in random_shape:
    for v in model_version:
        batch_size = sp[0]
        num_classes = 1000
        model_test = Model_vgg(v , num_classes)
        random_tensor = torch.rand(sp)
        output   = model_test(random_tensor)
        print(f'{v} model output shape :{output.shape}')
        assert tuple(output.shape) == (batch_size,num_classes)

A model output shape :torch.Size([9, 1000])
A_lrn model output shape :torch.Size([9, 1000])
B model output shape :torch.Size([9, 1000])
C model output shape :torch.Size([9, 1000])
D model output shape :torch.Size([9, 1000])
E model output shape :torch.Size([9, 1000])
A model output shape :torch.Size([1, 1000])
A_lrn model output shape :torch.Size([1, 1000])
B model output shape :torch.Size([1, 1000])
C model output shape :torch.Size([1, 1000])
D model output shape :torch.Size([1, 1000])
E model output shape :torch.Size([1, 1000])
A model output shape :torch.Size([1, 1000])
A_lrn model output shape :torch.Size([1, 1000])
B model output shape :torch.Size([1, 1000])
C model output shape :torch.Size([1, 1000])
D model output shape :torch.Size([1, 1000])
E model output shape :torch.Size([1, 1000])
A model output shape :torch.Size([8, 1000])
A_lrn model output shape :torch.Size([8, 1000])
B model output shape :torch.Size([8, 1000])
C model output shape :torch.Size([8, 1000])
D model output s